<a href="https://colab.research.google.com/github/lfauren/03MAIR-Algoritmos-de-Optimizacion-2019/blob/master/AG2/Luis%20Faur%C3%A9%20Navarro%20-%20AG2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Luis Fauré Navarro - AG2<br>
Actividad Guiada 2<br>
Url: https://github.com/lfauren/03MAIR-Algoritmos-de-Optimizacion-2019/tree/master/AG2

In [0]:
from time import time
#Función para calcular el tiempo de ejecución
def calcular_tiempo(f): 
    def wrapper(*args, **kwargs):      
        inicio = time()       
        resultado = f(*args, **kwargs)       
        tiempo = time() - inicio
        print("Tiempo de ejecución para algoritmo: "+str(tiempo))
        return resultado 
    return wrapper

In [2]:
import math
import random

N=1000
LISTA_2D = [(random.randrange(1,N*10),random.randrange(1,N*10)) for _ in range(N)]
print(LISTA_2D)

[(3595, 2047), (593, 7870), (4987, 1491), (3429, 1429), (385, 9620), (4940, 8885), (8809, 2954), (7008, 2857), (6117, 895), (4275, 9592), (1466, 8655), (4793, 90), (9091, 3885), (2218, 5071), (9423, 7527), (6131, 8411), (9145, 1944), (5069, 5520), (9719, 3269), (5938, 2796), (1732, 2392), (2382, 2598), (5504, 6701), (9809, 2977), (1172, 347), (6967, 8713), (5941, 268), (8528, 2888), (1588, 7390), (8098, 3362), (6811, 5580), (1130, 8022), (2731, 8828), (1564, 4800), (81, 9420), (5242, 6572), (2689, 4423), (3452, 1502), (2915, 2954), (8029, 3869), (5808, 9325), (9922, 7516), (5416, 6422), (8970, 421), (8187, 2924), (1162, 224), (9594, 4643), (2817, 7159), (8004, 1321), (2573, 8445), (7371, 8141), (1716, 3881), (567, 2569), (6665, 6386), (2669, 7871), (2676, 35), (1314, 1154), (3887, 7448), (8118, 740), (5240, 411), (7253, 8174), (5449, 42), (9489, 3913), (2443, 3166), (6321, 950), (9007, 3159), (4970, 2997), (8076, 5988), (867, 9709), (4266, 7418), (6101, 408), (6688, 2841), (9019, 3712)

In [1]:
def distancia(A,B):
  if type(A) is int or type(A) is float:
    return abs(B-A)
  else:
    return math.sqrt(sum([(A[i]-B[i])**2 for i in range(len(A))]))

distancia((1,3),(2,5))

2.23606797749979

In [6]:
# Fuerza bruta
@calcular_tiempo
def distancia_fuerza_bruta(L):
  mejor_distancia = 100000e10
  A,B = (),()
  for i in range(len(L)):
    for j in range(i+1,len(L)):
        if distancia(L[i],L[j]) < mejor_distancia:
          A,B = L[i],L[j]
  return A,B

distancia_fuerza_bruta(LISTA_2D)

Tiempo de ejecución para algoritmo: 1.1288862228393555


((986, 9587), (6460, 3359))

In [34]:
def distancia_divide_y_venceras(L):
  if len(L) < 10:
    return distancia_fuerza_bruta(L)
  #pivote = sum([L[i][0] for i in range(len(L))])/len(L)
  LISTA_IZQ = sorted(L, key=lambda x: x[0])[:len(L)//2]
  LISTA_DER = sorted(L, key=lambda x: x[0])[len(L)//2:]
  PUNTOS_LISTA_IZQ = distancia_divide_y_venceras(LISTA_IZQ)
  PUNTOS_LISTA_DER = distancia_divide_y_venceras(LISTA_DER)
  return distancia_fuerza_bruta(PUNTOS_LISTA_IZQ + PUNTOS_LISTA_DER) 

@calcular_tiempo
def LANZA(L):
  return distancia_divide_y_venceras(L)

SOL = LANZA(LISTA_2D[:15])
print(SOL)

Tiempo de ejecución para algoritmo: 0.00016617774963378906
Tiempo de ejecución para algoritmo: 7.867813110351562e-05
Tiempo de ejecución para algoritmo: 1.7642974853515625e-05
Tiempo de ejecución para algoritmo: 0.002064228057861328
((9091, 3885), (9423, 7527))


In [30]:
TARIFAS = [[0,5,4,3,999,999,999],
           [999,0,999,2,3,999,11],
           [999,999, 0,1,999,4,10],
           [999,999,999, 0,5,6,9],
           [999,999, 999,999,0,999,4],
           [999,999, 999,999,999,0,3],
           [999,999,999,999,999,999,0]]

def Precios(TARIFAS):
  N=len(TARIFAS[0])
  PRECIOS = [[9999]*N for i in [9999]*N]
  RUTAS = [[""]*N for i in [""]*N]
  for i in range(N-1):
    for j in range(i+1,N):
      MIN = TARIFAS[i][j]
      RUTAS[i][j] = i
      for k in range(i,j):
        if PRECIOS[i][k] + TARIFAS[k][j] < MIN:
          MIN = min(MIN, PRECIOS[i][k] + TARIFAS[k][j])
          RUTAS[i][j] = k
      PRECIOS[i][j] = MIN
  return PRECIOS, RUTAS

PRECIOS, RUTAS = Precios(TARIFAS)
print(PRECIOS)
print()
print(RUTAS)

def calcular_ruta(RUTAS, desde, hasta):
  if desde == hasta:
    return desde
  else:
    return str(calcular_ruta(RUTAS,desde,RUTAS[desde][hasta])) + ',' + str(RUTAS[desde][hasta])
  
print('\nLa ruta es:')
calcular_ruta(RUTAS,0,6)

[[9999, 5, 4, 3, 8, 8, 11], [9999, 9999, 999, 2, 3, 8, 7], [9999, 9999, 9999, 1, 6, 4, 7], [9999, 9999, 9999, 9999, 5, 6, 9], [9999, 9999, 9999, 9999, 9999, 999, 4], [9999, 9999, 9999, 9999, 9999, 9999, 3], [9999, 9999, 9999, 9999, 9999, 9999, 9999]]

[['', 0, 0, 0, 1, 2, 5], ['', '', 1, 1, 1, 3, 4], ['', '', '', 2, 3, 2, 5], ['', '', '', '', 3, 3, 3], ['', '', '', '', '', 4, 4], ['', '', '', '', '', '', 5], ['', '', '', '', '', '', '']]

La ruta es:


'0,0,2,5'